> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

---

# Leçon 5 : Mon premier LLM

## On assemble tout !

Tu as appris :
1. **Prédire la suite** avec des probabilités
2. **Apprendre de ses erreurs** avec la loss et le gradient
3. **Les embeddings** pour donner une mémoire au modèle
4. **L'attention** pour regarder les lettres importantes

Maintenant, on met tout ensemble pour créer un **vrai mini-LLM**
qui génère des noms de Pokémon inventés !

In [ ]:
from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 3


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_barres(valeurs, etiquettes, titre="Probabilites"):
    """Affiche des barres horizontales HTML."""
    rows = ""
    max_val = max(valeurs) if valeurs else 1
    for etiq, val in zip(etiquettes, valeurs, strict=False):
        pct = val / max_val * 100 if max_val > 0 else 0
        rows += (
            f'<tr><td style="padding:3px 8px;font-weight:bold;font-size:1em">{etiq}</td>'
            f'<td style="padding:3px;width:300px"><div style="background:linear-gradient(90deg,#3949ab,#6a1b9a);'
            f'width:{max(pct, 2):.0f}%;height:20px;border-radius:4px"></div></td>'
            f'<td style="padding:3px 8px;font-size:0.9em">{val:.0%}</td></tr>'
        )
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0"><b>{titre}</b>'
            f'<table style="border-collapse:collapse;margin-top:4px">{rows}</table></div>'
        )
    )


def afficher_attention(poids, positions, titre="Poids d'attention"):
    """Affiche les poids d'attention sous forme de barres horizontales."""
    rows = ""
    max_val = max(poids) if poids else 1
    for pos, w in zip(positions, poids, strict=False):
        pct = w / max_val * 100 if max_val > 0 else 0
        rows += (
            f'<tr><td style="padding:3px 8px;font-weight:bold;font-size:1em">{pos}</td>'
            f'<td style="padding:3px;width:300px"><div style="background:linear-gradient(90deg,#3949ab,#6a1b9a);'
            f'width:{max(pct, 2):.0f}%;height:20px;border-radius:4px"></div></td>'
            f'<td style="padding:3px 8px;font-size:0.9em">{w:.1%}</td></tr>'
        )
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0"><b>{titre}</b>'
            f'<table style="border-collapse:collapse;margin-top:4px">{rows}</table></div>'
        )
    )


def afficher_architecture():
    """Affiche le schema de reseau du mini-LLM avec connexions residuelles."""
    # Styles communs
    box = (
        "padding:8px 12px;border-radius:8px;text-align:center;"
        "font-family:system-ui,-apple-system,sans-serif;position:relative;z-index:1"
    )
    dim = "font-size:0.75em;color:#888;font-style:italic"
    arrow = "font-size:1.2em;color:#999"

    html = (
        '<!-- tuto-viz --><div style="margin:8px 0"><b>Architecture du mini-LLM</b>'
        '<div style="display:flex;flex-direction:column;align-items:center;'
        'gap:2px;margin-top:8px;position:relative">'
        # Entree
        f'<div style="background:#f8f9fa;border:2px solid #495057;{box};width:320px">'
        '<b style="color:#495057">\U0001f4e5 Entree</b><br>'
        f'<span style="{dim}">".pik" \u2192 [0, 16, 9, 11]</span></div>'
        f'<div style="{arrow}">\u25bc</div>'
        # Embeddings
        f'<div style="background:#e8f5e9;border:2px solid #2e7d32;{box};width:320px">'
        '<b style="color:#2e7d32">\U0001f3af Token + Position Embedding</b><br>'
        f'<span style="{dim}">tok_emb[id] + pos_emb[pos] \u2192 [n, {"{EMBED_DIM}"}]</span></div>'
        f'<div style="{arrow}">\u25bc</div>'
    )

    # Self-Attention block avec detail Q/K/V + skip connection
    html += (
        '<div style="position:relative;width:380px">'
        # Skip connection gauche (ligne verticale qui contourne l'attention)
        '<div style="position:absolute;left:0;top:0;bottom:0;width:30px">'
        '<div style="border-left:2px dashed #1565c0;border-top:2px dashed #1565c0;'
        "border-bottom:2px dashed #1565c0;height:100%;margin-left:10px;"
        'border-radius:8px 0 0 8px"></div></div>'
        '<div style="position:absolute;left:2px;top:50%;transform:translateY(-50%) rotate(90deg);'
        f'font-size:0.65em;color:#1565c0;white-space:nowrap">\u2795 residuelle</div>'
        # Boite attention
        '<div style="margin-left:35px;margin-right:5px">'
        f'<div style="background:#e3f2fd;border:2px solid #1565c0;{box}">'
        '<b style="color:#1565c0">\U0001f50d Self-Attention</b><br>'
        f'<span style="{dim}">Q = Wq \u00d7 x, K = Wk \u00d7 x, V = Wv \u00d7 x<br>'
        "scores = Q \u00b7 K / \u221ad \u2192 softmax \u2192 \u2211 poids \u00d7 V</span>"
        "</div></div></div>"
        f'<div style="{arrow}">\u25bc + skip</div>'
    )

    # MLP block avec skip connection
    html += (
        '<div style="position:relative;width:380px">'
        # Skip connection gauche (ligne verticale qui contourne le MLP)
        '<div style="position:absolute;left:0;top:0;bottom:0;width:30px">'
        '<div style="border-left:2px dashed #e65100;border-top:2px dashed #e65100;'
        "border-bottom:2px dashed #e65100;height:100%;margin-left:10px;"
        'border-radius:8px 0 0 8px"></div></div>'
        '<div style="position:absolute;left:2px;top:50%;transform:translateY(-50%) rotate(90deg);'
        f'font-size:0.65em;color:#e65100;white-space:nowrap">\u2795 residuelle</div>'
        # Boite MLP
        '<div style="margin-left:35px;margin-right:5px">'
        f'<div style="background:#fff3e0;border:2px solid #e65100;{box}">'
        '<b style="color:#e65100">\U0001f9e0 MLP (ReLU)</b><br>'
        f'<span style="{dim}">W1 \u00d7 x + b1 \u2192 ReLU \u2192 W2 \u00d7 h + b2</span>'
        "</div></div></div>"
        f'<div style="{arrow}">\u25bc + skip</div>'
    )

    # Projection + Softmax + Sortie
    html += (
        f'<div style="background:#fce4ec;border:2px solid #c62828;{box};width:320px">'
        '<b style="color:#c62828">\U0001f4ca Projection + Softmax</b><br>'
        f'<span style="{dim}">W_out \u00d7 x \u2192 logits \u2192 softmax \u2192 probas</span></div>'
        f'<div style="{arrow}">\u25bc</div>'
        f'<div style="background:#f3e5f5;border:2px solid #6a1b9a;{box};width:320px">'
        '<b style="color:#6a1b9a">\U0001f4e4 Sortie</b><br>'
        f'<span style="{dim}">Probabilite pour chaque lettre (27 valeurs)</span></div>'
        "</div>"
        '<div style="margin-top:8px;color:#555;font-size:0.92em;text-align:center">'
        "C'est la meme architecture que GPT-2/3/4, juste en beaucoup plus petit.<br>"
        '<span style="color:#1565c0">Les lignes pointillees</span> sont les '
        "<b>connexions residuelles</b> (raccourcis qui aident l'apprentissage)."
        "</div></div>"
    )
    display(HTML(html))


print("Outils de visualisation charges !")

---
## Architecture de notre LLM

Execute la cellule ci-dessous pour voir l'architecture de notre mini-LLM.

In [ ]:
import math
import random

random.seed(42)

# Configuration
VOCAB = list(".abcdefghijklmnopqrstuvwxyz")
VOCAB_SIZE = len(VOCAB)  # 27
EMBED_DIM = 16  # taille des embeddings
CONTEXT = 8  # nombre max de lettres en contexte
NUM_HEADS = 1  # tête d'attention (simplifiée pour la clarté)
HEAD_DIM = EMBED_DIM // NUM_HEADS  # = 16 (identique à EMBED_DIM avec 1 tête)
HIDDEN_DIM = 32  # taille du MLP

char_to_id = {c: i for i, c in enumerate(VOCAB)}
id_to_char = {i: c for i, c in enumerate(VOCAB)}

print("Configuration du mini-LLM :")
print(f"  Vocabulaire : {VOCAB_SIZE} caractères")
print(f"  Dimension embeddings : {EMBED_DIM}")
print(f"  Contexte max : {CONTEXT} lettres")
print(f"  Têtes d'attention : {NUM_HEADS}")
print(f"  Dimension par tête : {HEAD_DIM}")
print(f"  Taille MLP : {HIDDEN_DIM}")

# Comptons les paramètres
nb_params = (
    VOCAB_SIZE * EMBED_DIM  # token embeddings       = 27*16 = 432
    + CONTEXT * EMBED_DIM  # position embeddings    = 8*16  = 128
    + 3 * EMBED_DIM * EMBED_DIM  # Q, K, V              = 3*256 = 768
    + EMBED_DIM * HIDDEN_DIM  # MLP couche 1 (poids)  = 16*32 = 512
    + HIDDEN_DIM  # MLP couche 1 (biais)  = 32
    + HIDDEN_DIM * EMBED_DIM  # MLP couche 2 (poids)  = 32*16 = 512
    + EMBED_DIM  # MLP couche 2 (biais)  = 16
    + EMBED_DIM * VOCAB_SIZE  # couche de sortie      = 16*27 = 432
)
print(f"  Nombre de paramètres : {nb_params:,}")
print()
print(f"  (GPT-4 en a ~1,800,000,000,000 -- {nb_params / 1.8e12 * 100:.10f}% de GPT-4)")

# Visualisation de l'architecture
afficher_architecture()

In [ ]:
exercice(
    1,
    "Change la taille du modele",
    "Change <code>EMBED_DIM_test</code> ci-dessous (essaie 8 ou 32), puis <b>Shift + Entree</b>.",
    "Doubler EMBED_DIM quadruple presque le nombre de parametres !",
)

# ==== MODIFIE ICI ====
EMBED_DIM_test = 16  # <-- Essaie 8 (petit) ou 32 (grand) !
# ======================

HIDDEN_DIM_test = EMBED_DIM_test * 2

nb_params_test = (
    VOCAB_SIZE * EMBED_DIM_test  # token embeddings
    + CONTEXT * EMBED_DIM_test  # position embeddings
    + 3 * EMBED_DIM_test * EMBED_DIM_test  # Q, K, V
    + EMBED_DIM_test * HIDDEN_DIM_test  # MLP couche 1 (poids)
    + HIDDEN_DIM_test  # MLP couche 1 (biais)
    + HIDDEN_DIM_test * EMBED_DIM_test  # MLP couche 2 (poids)
    + EMBED_DIM_test  # MLP couche 2 (biais)
    + EMBED_DIM_test * VOCAB_SIZE  # sortie
)

print(f"Avec EMBED_DIM = {EMBED_DIM_test} :")
print(f"  Nombre de parametres : {nb_params_test:,}")
print(f"  (Notre modele en a {nb_params:,})")
if EMBED_DIM_test < EMBED_DIM:
    print("  -> Plus petit : moins de parametres, plus rapide, mais moins precis.")
elif EMBED_DIM_test > EMBED_DIM:
    print(
        "  -> Plus grand : plus de parametres, plus lent, mais potentiellement meilleur."
    )

# Validation exercice 1
verifier(
    1,
    EMBED_DIM_test != 16,
    f"Bien joue ! Avec {EMBED_DIM_test} dimensions, le modele a {nb_params_test:,} parametres.",
    "Change EMBED_DIM_test pour une autre valeur, par exemple 8 ou 32.",
)

In [ ]:
# Fonctions utilitaires


def rand_matrix(rows, cols, scale=0.3):
    return [[random.gauss(0, scale) for _ in range(cols)] for _ in range(rows)]


def rand_vector(size, scale=0.3):
    return [random.gauss(0, scale) for _ in range(size)]


def mat_vec(mat, vec):
    """Multiplication matrice x vecteur."""
    return [sum(mat[i][j] * vec[j] for j in range(len(vec))) for i in range(len(mat))]


def vec_add(a, b):
    return [x + y for x, y in zip(a, b, strict=False)]


def softmax(scores):
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]


def relu(x):
    """Si positif, on garde. Si négatif, on met à zéro."""
    return [max(0, v) for v in x]


print("Fonctions utilitaires définies !")

In [ ]:
# Initialiser tous les poids du modèle

# Embeddings
tok_emb = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.5)  # token -> vecteur
pos_emb = rand_matrix(CONTEXT, EMBED_DIM, 0.5)  # position -> vecteur

# Attention (1 tête pour la clarté)
Wq = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
Wk = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
Wv = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)

# MLP
W1 = rand_matrix(HIDDEN_DIM, EMBED_DIM, 0.2)
b1 = rand_vector(HIDDEN_DIM, 0.1)
W2 = rand_matrix(EMBED_DIM, HIDDEN_DIM, 0.2)
b2 = rand_vector(EMBED_DIM, 0.1)

# Sortie
W_out = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.2)

print("Modèle initialisé avec des poids aléatoires.")
print("Il ne sait rien encore -- il faut l'entraîner !")

In [ ]:
def forward_llm(sequence_ids):
    """Passe une séquence dans le mini-LLM et retourne les probas pour le prochain token."""
    n = len(sequence_ids)

    # 1. Embeddings : token + position
    hidden = []
    for i, tok_id in enumerate(sequence_ids):
        h = vec_add(tok_emb[tok_id], pos_emb[i % CONTEXT])
        hidden.append(h)

    # 2. Self-Attention (sur la dernière position)
    # Query pour la dernière position
    q = mat_vec(Wq, hidden[-1])

    # Keys et Values pour toutes les positions
    scores = []
    values = []
    for i in range(n):
        k = mat_vec(Wk, hidden[i])
        v = mat_vec(Wv, hidden[i])
        score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
        scores.append(score)
        values.append(v)

    attn_weights = softmax(scores)

    # Somme pondérée des values
    attn_out = [0.0] * EMBED_DIM
    for i in range(n):
        for d in range(EMBED_DIM):
            attn_out[d] += attn_weights[i] * values[i][d]

    # Connexion résiduelle
    x = vec_add(hidden[-1], attn_out)

    # 3. MLP
    h = relu(vec_add(mat_vec(W1, x), b1))
    mlp_out = vec_add(mat_vec(W2, h), b2)

    # Connexion résiduelle
    x = vec_add(x, mlp_out)

    # 4. Sortie : scores pour chaque lettre
    logits = mat_vec(W_out, x)
    probas = softmax(logits)

    return probas


def _calculer_poids_attention(texte):
    """Calcule les poids d'attention pour un texte (utilise les poids globaux)."""
    ids = [char_to_id[c] for c in texte]
    hidden = [vec_add(tok_emb[tid], pos_emb[i % CONTEXT]) for i, tid in enumerate(ids)]
    q = mat_vec(Wq, hidden[-1])
    scores = []
    for i in range(len(ids)):
        k = mat_vec(Wk, hidden[i])
        score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
        scores.append(score)
    return softmax(scores)


# Test avant entraînement
test_ids = [char_to_id[c] for c in ".pik"]
probas = forward_llm(test_ids)
top5 = sorted(range(VOCAB_SIZE), key=lambda i: -probas[i])[:5]

print("Avant entraînement -- prédictions après '.pik' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print("(La bonne réponse serait 'a' pour 'pikachu')")

# Visualisation des top-5 predictions
afficher_barres(
    [probas[i] for i in top5],
    [id_to_char[i] for i in top5],
    titre="Top 5 predictions apres '.pik' (avant entrainement)",
)
# Visualisation des poids d'attention
_attn_viz = _calculer_poids_attention(".pik")
afficher_attention(_attn_viz, list(".pik"), titre="Poids d'attention pour '.pik'")

In [ ]:
exercice(
    2,
    "Change le debut du mot",
    'Change <code>mon_debut</code> ci-dessous (essaie <code>".bul"</code>, <code>".evo"</code> ou <code>".dra"</code>).',
    "Le modele predit des lettres differentes selon le debut.",
)

# ==== MODIFIE ICI ====
mon_debut = ".pik"  # <-- Essaie ".bul", ".evo" ou ".dra" !
# ======================

test_ids = [char_to_id[c] for c in mon_debut]
probas = forward_llm(test_ids)
top5 = sorted(range(VOCAB_SIZE), key=lambda i: -probas[i])[:5]

print(f"Predictions apres '{mon_debut}' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print()
print("(Le modele n'est pas entraine, donc c'est du hasard !)")
print("(Apres entrainement dans la lecon 6, les predictions seront meilleures.)")

# Visualisation des predictions
afficher_barres(
    [probas[i] for i in top5],
    [id_to_char[i] for i in top5],
    titre=f"Top 5 predictions apres '{mon_debut}'",
)

# Visualisation des poids d'attention
_attn = _calculer_poids_attention(mon_debut)
afficher_attention(
    _attn, list(mon_debut), titre=f"Poids d'attention pour '{mon_debut}'"
)

# Validation exercice 2
verifier(
    2,
    mon_debut != ".pik",
    f"Super ! Tu as teste les predictions apres '{mon_debut}'.",
    "Change mon_debut pour un autre debut, par exemple '.bul' ou '.evo'.",
)

In [ ]:
# Entraînement simplifié
# (On utilise une méthode numérique pour les gradients,
#  plus lente mais plus facile à comprendre)

pokemons = [
    "arcanin",
    "bulbizarre",
    "carapuce",
    "dracaufeu",
    "ectoplasma",
    "evoli",
    "felinferno",
    "gardevoir",
    "goupix",
    "lokhlass",
    "lucario",
    "metamorph",
    "mewtwo",
    "noctali",
    "pikachu",
    "rondoudou",
    "ronflex",
    "salameche",
    "togepi",
    "voltali",
]


def calcul_loss(pokemons):
    """Calcule la loss moyenne sur tous les Pokémon."""
    loss_totale = 0
    nb = 0
    for pokemon in pokemons:
        mot = "." + pokemon + "."
        ids = [char_to_id[c] for c in mot]
        for i in range(1, len(ids)):
            seq = ids[:i]
            cible = ids[i]
            probas = forward_llm(seq[-CONTEXT:])
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1
    return loss_totale / nb


loss_initiale = calcul_loss(pokemons)
print(f"Loss initiale : {loss_initiale:.3f}")
print(f"(Loss d'un modèle parfaitement aléatoire : {math.log(VOCAB_SIZE):.3f})")
print()
print("L'entraînement complet prendrait du temps en Python pur.")
print("C'est pour ça qu'en vrai on utilise PyTorch avec des GPU !")
print()
print("Mais l'ARCHITECTURE est exactement la même que GPT-2, GPT-3, GPT-4.")
print("Seuls la taille et la puissance de calcul changent.")

In [ ]:
# Génération (même sans entraînement complet, on peut voir le mécanisme)


def generer_llm(debut=".", temperature=1.0, max_len=15):
    """Génère un nom de Pokémon lettre par lettre avec notre mini-LLM."""
    ids = [char_to_id[c] for c in debut]
    resultat = debut

    for _ in range(max_len):
        probas = forward_llm(ids[-CONTEXT:])

        # Température : < 1 = plus conservateur, > 1 = plus créatif
        if temperature != 1.0:
            logits = [math.log(p + 1e-10) / temperature for p in probas]
            probas = softmax(logits)

        # Choisir la prochaine lettre
        idx = random.choices(range(VOCAB_SIZE), weights=probas, k=1)[0]

        if idx == char_to_id["."]:
            break

        ids.append(idx)
        resultat += id_to_char[idx]

    return resultat[1:] if resultat.startswith(".") else resultat


print(
    "Noms de Pokémon générés (modèle non-entraîné, juste pour montrer le mécanisme) :"
)
print()
for _ in range(10):
    p = generer_llm(temperature=0.8)
    print(f"  {p.capitalize()}")

print()
print("C'est du charabia car le modèle n'est pas entraîné.")
print("Mais le MÉCANISME est exactement celui de ChatGPT !")

In [ ]:
exercice(
    3,
    "Joue avec la temperature",
    "Change <code>ma_temperature</code> ci-dessous (essaie 0.1 ou 2.0).",
    "Temperature basse = repetitif. Temperature haute = creatif.",
)

# ==== MODIFIE ICI ====
ma_temperature = 0.8  # <-- Essaie 0.1 (sage) ou 2.0 (fou) !
# ======================

print(f"Generation avec temperature = {ma_temperature} :")
print()
for _ in range(10):
    p = generer_llm(temperature=ma_temperature)
    print(f"  {p.capitalize()}")
print()
if ma_temperature < 0.5:
    print(
        "Temperature basse : le modele choisit toujours les lettres les plus probables."
    )
elif ma_temperature > 1.5:
    print("Temperature haute : le modele explore des combinaisons inhabituelles !")
else:
    print("Temperature moyenne : un bon equilibre entre creativite et coherence.")

# Validation exercice 3
verifier(
    3,
    ma_temperature != 0.8,
    f"Genial ! Tu as explore la temperature {ma_temperature}.",
    "Change ma_temperature pour une autre valeur, par exemple 0.1 ou 2.0.",
)

---
## Ce qu'on a appris

```
Leçon 1 : Compter les lettres qui suivent        -> bigramme
Leçon 2 : Apprendre de ses erreurs                -> entraînement
Leçon 3 : Regarder plusieurs lettres en arrière   -> embeddings + contexte
Leçon 4 : Choisir les lettres importantes          -> attention
Leçon 5 : Assembler le tout                       -> mini-LLM !
```

## La différence avec ChatGPT

| | Notre mini-LLM | ChatGPT |
|---|---|---|
| Architecture | La même ! | La même ! |
| Paramètres | ~2,800 | ~1,800,000,000,000 |
| Données | 20 Pokémon | Internet entier |
| Calcul | 1 PC, secondes | Des milliers de GPU, des mois |
| Résultat | Noms de Pokémon inventés | Conversations, code, poésie... |

L'algorithme est **le même**. La seule différence, c'est l'échelle.

> *"This file is the complete algorithm. Everything else is just efficiency."*
> -- Andrej Karpathy

---
## Pour aller plus loin

- **microgpt.py** : Le code complet de Karpathy avec l'autograd et l'entraînement
  [Lien](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95)

- **Vidéo "Let's build GPT"** : Karpathy explique tout en 2h
  [YouTube](https://www.youtube.com/watch?v=kCc8FmEb1nY)

- **nanoGPT** : Version avec PyTorch, entraînable pour de vrai
  [GitHub](https://github.com/karpathy/nanoGPT)

---

**Félicitations ! Tu as compris comment fonctionne un LLM.**

*Prochaine leçon : [06 - Entraîner le modèle](06_entrainer_le_modele.ipynb)*

---

### Sources (ISO 42001)

- **Architecture complète GPT (embedding + attention + MLP + softmax)** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy
- **Comparaison des paramètres GPT-4** : estimations publiques basées sur les rapports techniques OpenAI
- **Explication du forward pass complet** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023)
- **Concept de température pour la génération** : même source, section sampling
- **"Attention Is All You Need"** : Vaswani et al., 2017, [arXiv:1706.03762](https://arxiv.org/abs/1706.03762)
- **Dataset Pokémon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage éducatif. Source : [PokéAPI](https://pokeapi.co/)